In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from functions import *
from scipy.stats import gaussian_kde
from matplotlib_venn import venn2, venn3
import seaborn as sns
from scipy import stats

In [5]:
import torch
import torch.optim as optim
from torch import nn
import torch.nn.functional as F
import esm

In [7]:
#Load esm2
device = torch.device('cpu')
esm2_model, alphabet = esm.pretrained.esm2_t6_8M_UR50D() # 6 layers
esm2_model = esm2_model.to(device)
esm2_batch_converter = alphabet.get_batch_converter()

In [9]:
test_data = pd.read_csv('../data/Topt/test.csv')
ids = list(test_data.index)[:5]; seqs = list(test_data['sequence'])[:5]

In [10]:
inputs = [(ids[i], seqs[i]) for i in range(len(ids))]
batch_labels, batch_strs, batch_tokens = esm2_batch_converter(inputs)
batch_tokens = batch_tokens.to(device=device, non_blocking=True)
with torch.no_grad():
    emb = esm2_model(batch_tokens, repr_layers=[6], return_contacts=False)
emb = emb["representations"][6]
emb = emb.transpose(1,2)
emb = emb.to(device)

In [15]:
x_cnn = nn.Conv1d(320, 320, kernel_size=2*5+1, padding=5) 